In [38]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import timeseries_dataset_from_array
import matplotlib.pyplot as plt
import seaborn as sns
from MultiSeriesWindowsGenerator import MultiSeriesWindowsGenerator
import IPython.display

In [39]:
pd.set_option("display.max_row", 200)

In [40]:
df = pd.read_csv("data/all_data_aggr_nonan.csv", index_col=0)

In [41]:
LABELS = ['mood']
REGRESSORS = ['weekday', 'circumplex.arousal', 'circumplex.valence',
              'activity', 'screen', 'call', 'sms', 'appCat.builtin',
              'appCat.communication', 'appCat.entertainment', 'appCat.finance',
              'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social',
              'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']

DATE = 'days'  # always correct
IN_STEPS = 20  # use 7 days
OUT_STEPS = 1  # to predict 1 day in the future
GROUPBY = ['subject_id']
BATCH_SIZE = 8

In [42]:
n = len(df)
train_series = df.groupby(GROUPBY, as_index=False, group_keys=False).apply(
    lambda x: x.iloc[:int(len(x) * 0.7)]).reset_index(drop=True)
val_series = df.groupby(GROUPBY, as_index=False, group_keys=False).apply(
    lambda x: x.iloc[int(len(x) * 0.7):int(len(x) * 0.9)]).reset_index(drop=True)
test_series = df.groupby(GROUPBY, as_index=False, group_keys=False).apply(
    lambda x: x.iloc[int(len(x) * 0.9):]).reset_index(drop=True)

test_window = MultiSeriesWindowsGenerator(
    input_width=IN_STEPS, label_width=OUT_STEPS, shift=1, batch_size=BATCH_SIZE, GROUPBY=GROUPBY,
    label_columns=LABELS, regressor_columns=REGRESSORS, DATE=DATE, LABELS=LABELS)

test_window.update_datasets(train_series, val_series, test_series, norm=False)

In [43]:
a, b = test_window.example


In [44]:
lstm_model_classifier = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

MAX_EPOCHS = 5

In [45]:
def compile_and_fit_classifier(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min')

    # model.compile(loss=tf.keras.losses.MeanSquaredError(),
    #               optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1),
    #               metrics=[tf.keras.metrics.MeanAbsoluteError()])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val,
                        callbacks=[early_stopping])
    return history

In [46]:
history = compile_and_fit_classifier(lstm_model_classifier, test_window)
val_performance = {}
performance = {}

probs = lstm_model_classifier.predict(test_window.val)
preds = probs.argmax(axis=-1)
val_performance['LSTM'] = lstm_model_classifier.evaluate(test_window.val, verbose=0)

probs = lstm_model_classifier.predict(test_window.test)
preds = probs.argmax(axis=-1)
performance['LSTM'] = lstm_model_classifier.evaluate(test_window.test, verbose=0)

Epoch 1/5
61/61 [==============================] - 12s 65ms/step - loss: 1.3312 - sparse_categorical_accuracy: 0.4300 - val_loss: 0.4369 - val_sparse_categorical_accuracy: 0.8056
Epoch 2/5
61/61 [==============================] - 3s 35ms/step - loss: 0.9807 - sparse_categorical_accuracy: 0.4362 - val_loss: 0.4140 - val_sparse_categorical_accuracy: 0.8056
Epoch 3/5
61/61 [==============================] - 3s 42ms/step - loss: 0.9197 - sparse_categorical_accuracy: 0.4588 - val_loss: 0.4227 - val_sparse_categorical_accuracy: 0.7963
Epoch 4/5
24/24 [==============================] - 2s 15ms/step


In [ ]:
test_window.plot(lstm_model_classifier, single=True, max_subplots=10)

In [ ]:
a, b = test_window.example
lstm_model_classifier(a)